
# Genie API 応用編（質問パターンを変えてみる）

このノートブックでは、次の流れを確認します。

1. Genie に **違う内容の自然言語の質問を2つ** 送る（それぞれ新しい会話として開始）
2. 処理が完了するまで待ち、final_message を取得する
3. 各質問ごとに、生成された「説明テキスト」「SQL」「結果データセット」を取り出して比較する

In [0]:
%run ./_init_genie_api

### 1. 共通の取り出し関数を定義

In [0]:
def run_genie_question(question: str):
    """
    質問を1つ投げて、final_message から
    - 説明テキスト
    - 生成SQL
    - 結果DataFrame
    を取得して返すユーティリティ。
    """
    print("=== 質問 ===")
    print(question)

    # 会話開始
    start_resp = start_conversation(question)
    conversation_id = start_resp.get("conversation", {}).get("id")
    message_id = start_resp.get("message", {}).get("id")

    print("\n=== 会話情報 ===")
    print(f"conversation_id: {conversation_id}")
    print(f"message_id     : {message_id}")

    # 完了まで待つ
    final_message = wait_until_completed(
        conversation_id,
        message_id,
        timeout_sec=120,    # 2分でタイムアウト
        interval_sec=5,
    )

    print("\n=== final_message（status / content） ===")
    print(f"status : {final_message.get('status')}")
    print(f"content: {final_message.get('content')}")

    # attachments から説明テキスト / SQL / attachment_id / 提案質問を取り出す
    attachments = final_message.get("attachments", [])

    answer_text = None
    sql_text = None
    attachment_id = None
    suggested_questions = []

    for i, att in enumerate(attachments):
        print(f"--- attachment #{i} --- keys: {list(att.keys())}")

        query_part = att.get("query")
        suggested_part = att.get("suggested_questions")

        if query_part and sql_text is None:
            sql_text = query_part.get("statement") or query_part.get("query")
            answer_text = query_part.get("description")
            attachment_id = att.get("attachment_id") or query_part.get("attachment_id")

        if suggested_part:
            qs = suggested_part.get("questions") or []
            suggested_questions.extend(qs)

    print("\n=== Genie の説明テキスト ===")
    print(answer_text)

    print("\n=== 生成されたSQL ===")
    print(sql_text)

    print("\n=== クエリ結果用 attachment_id ===")
    print(attachment_id)

    print("\n=== 提案質問 ===")
    for q in suggested_questions:
        print(f"  - {q}")

    # 結果DataFrameを取得
    df_result = None
    if attachment_id:
        result_json = get_query_result(
            conversation_id=conversation_id,
            message_id=message_id,
            attachment_id=attachment_id,
        )
        df_result = genie_query_result_to_df(result_json)

        print("\n=== クエリ結果（先頭20行） ===")
        display(df_result.limit(20))
    else:
        print("クエリ結果用 attachment_id がありませんでした。")

    return {
        "conversation_id": conversation_id,
        "message_id": message_id,
        "answer_text": answer_text,
        "sql": sql_text,
        "attachment_id": attachment_id,
        "suggested_questions": suggested_questions,
        "df_result": df_result,
    }

### 2. 質問A: トップ10商品

In [0]:
info_a = run_genie_question(
    question="直近30日間の売上トップ10の商品を教えて",
)

### 3. 質問B: トップ10商品のカテゴリ別売上合計

In [0]:
info_b = run_genie_question(
    question="直近30日間の売上トップ10商品のカテゴリ別の売上合計を教えて",
)